# Add SpatialData

- Convert the cohort of fovs and labels into several spatialdata objets, one `spatialdata` object per FOV.
- Add metadata to the `spatialdata` objects and save them to LaminDB as artifacts.

## Setup


In [ ]:
import bionty as bt
import lamindb as ln
from distributed import Client, LocalCluster
from upath import UPath

import nbl

cluster = LocalCluster(n_workers=10)
client = Client(cluster)

In [ ]:
client

In [3]:
ln.settings.sync_git_repo = "https://github.com/karadavis-lab/nbl.git"

In [ ]:
ln.track(project="Neuroblastoma")

In [5]:
bt.settings.organism = "human"

In [6]:
raw_data_dir = UPath("../../data/raw/nbl_cohort")

fov_dir = raw_data_dir / "images"
label_dir = raw_data_dir / "segmentation" / "labels"

In [ ]:
hu_sdatas = nbl.io.convert_cohort(
    fov_dir=fov_dir,
    label_dir=label_dir,
    filter_fovs=r"Hu-*",
    return_collection=False,
)

In [ ]:
nbl_sdatas = nbl.io.convert_cohort(
    fov_dir=fov_dir,
    label_dir=label_dir,
    filter_fovs=r"NBL-*",
    return_collection=False,
)

In [11]:
nbl_sdata = nbl_sdatas["NBL-1-R5C8"]

# ln.Artifact.from_spatialdata(hu_sdata, key="control/Hu-Adrenal-Medulla-R5C12.zarr")

In [9]:
clinical_data = ln.Artifact.filter(ln.Q(key__contains="clinical_data")).one().load()

In [ ]:
nbl_a = ln.Artifact.from_spatialdata(nbl_sdata, key="NBL-1-R5C8.zarr")

In [ ]:
clinical_data[clinical_data["FOV"] == "NBL-1-R5C8"]

In [ ]:
ln.curators.SpatialDataCurator

In [ ]:
for _, fov_row in clinical_data.iterrows():
    # fov = fov_row["FOV"]
    fov_name = fov_row["FOV"]
    fov_sdata = nbl_sdatas[fov_name]

    fov_sdata_artifact = ln.Artifact.from_spatialdata(fov_sdata, key=f"nbl_sdatas/{fov_name}.zarr")
    fov_sdata_artifact.save()

    for f in (features := ln.Feature.lookup()):
        match f:
            case ln.Feature():
                feature_name = f.name
            case _:
                continue
        feature_value = fov_row[feature_name]
        feature = ln.Feature.filter(ln.Q(name=feature_name)).one()
        fov_sdata_artifact.labels.add(records=ln.ULabel.filter(ln.Q(name=feature_value)), feature=feature)

    # nbl_a.labels.add(records = ln.ULabel(name=feature_name, value=feature_value), feature=f)

INFO     The Zarr backing store has been changed from                                                              
         /Users/srivarra/Library/Caches/lamindb/gnKmLGXiaUgQswQR0000.zarr the new file path:                       
         /Users/srivarra/Library/Caches/lamindb/r1eG0oZIhYpPCCs00000.zarr                                          
→ returning existing artifact with same hash: Artifact(uid='gnKmLGXiaUgQswQR0000', is_latest=True, key='nbl_sdatas/NBL-1-R5C8.zarr', suffix='.zarr', kind='dataset', otype='SpatialData', size=71691210, hash='lKpzDK0t_3EiRojqioa4BA', n_files=64, space_id=1, storage_id=1, run_id=3, created_by_id=1, created_at=2025-05-03 19:48:05 UTC); to track this artifact as an input, use: ln.Artifact.get()
INFO     The Zarr backing store has been changed from                                                              
         /Users/srivarra/Library/Caches/lamindb/ZiOctB0cidVaHwQI0000.zarr the new file path:                       
         /Users/srivarra/Li

In [63]:
ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas/NBL-7-R13C12.zarr")).latest_version().one().describe()

Artifact .zarr/SpatialData
├── General
│   ├── .uid = 'SEkev9gyrAhdPSNh0000'
│   ├── .key = 'nbl_sdatas/NBL-7-R13C12.zarr'
│   ├── .size = 68085136
│   ├── .hash = 'G17MZGkZAXz5aBNDooAM1A'
│   ├── .n_files = 64
│   ├── .path = /Users/srivarra/Davis Lab/nbl/data/nbl-dataset/.lamindb/SEkev9gyrAhdPSNh.zarr
│   ├── .created_by = srivarra
│   ├── .created_at = 2025-05-03 12:49:21
│   └── .transform = 'Add SpatialData'
├── Linked features
│   └── 11q LOH                     cat[ULabel]                No                                       
│       17q gain                    cat[ULabel]                Yes                                      
│       1p LOH                      cat[ULabel]                Yes                                      
│       7q gain                     cat[ULabel]                No                                       
│       ALK                         cat[ULabel]                F1174L                                   
│       Classification              cat[ULabel]                Post-Chemotherapy                        
│       Degree of differentiation   cat[ULabel]                Poorly Differentiated                    
│       FOV                         cat[ULabel]                NBL-7-R13C12                             
│       Genomic studies done        cat[ULabel]                SNP array|ALK/PHOX2B sequencing          
│       Genomics source             cat[ULabel]                This specimen                            
│       Histologic classification…  cat[ULabel]                Unfavorable                              
│       INRG stage                  cat[ULabel]                M                                        
│       MKI                         cat[ULabel]                High                                     
│       Ploidy value                cat[ULabel]                Diploid                                  
│       Risk                        cat[ULabel]                High                                     
│       Sex                         cat[ULabel]                Female                                   
│       UID                         cat[ULabel]                PC19-MARI-419-7                          
└── Labels
    └── .projects                   Project                    Neuroblastoma                            
        .ulabels                    ULabel                     NBL-7-R13C12, PC19-MARI-419-7, Post-Chem…

In [27]:
ln.ULabel.filter(ln.Q(name=feature_value))

<QuerySet [ULabel(uid='obzUy9nz', name='NBL-1-R5C8', is_type=False, space_id=1, created_by_id=1, run_id=2, created_at=2025-04-29 21:31:43 UTC)]>

In [47]:
ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas"))

<QuerySet [Artifact(uid='gnKmLGXiaUgQswQR0000', is_latest=True, key='nbl_sdatas/NBL-1-R5C8.zarr', suffix='.zarr', kind='dataset', otype='SpatialData', size=71691210, hash='lKpzDK0t_3EiRojqioa4BA', n_files=64, space_id=1, storage_id=1, run_id=3, created_by_id=1, created_at=2025-05-03 19:48:05 UTC), Artifact(uid='ZiOctB0cidVaHwQI0000', is_latest=True, key='nbl_sdatas/NBL-1-R6C8.zarr', suffix='.zarr', kind='dataset', otype='SpatialData', size=74604876, hash='xHr_IS9g-nBkzCxEG4vHWQ', n_files=64, space_id=1, storage_id=1, run_id=3, created_by_id=1, created_at=2025-05-03 19:48:06 UTC), Artifact(uid='jr7ZloWSZuhO6n5G0000', is_latest=True, key='nbl_sdatas/NBL-1-R12C8.zarr', suffix='.zarr', kind='dataset', otype='SpatialData', size=163135752, hash='ko_Uo4bXrlxJW4VsmrPJhA', n_files=64, space_id=1, storage_id=1, run_id=3, created_by_id=1, created_at=2025-05-03 19:48:06 UTC), Artifact(uid='RFSK8HEVxxQxiYeZ0000', is_latest=True, key='nbl_sdatas/NBL-1-R13C8.zarr', suffix='.zarr', kind='dataset', oty

In [55]:
ln.Collection(ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas")))

FieldValidationError: ['key'] are required.

In [60]:
ln.Collection(artifacts=ln.Artifact.filter(ln.Q(key__contains="nbl_sdatas")).latest_version(), key="nbl fovs")

Collection(uid='wsvEs3A9IwuleT2J0000', is_latest=True, key='nbl fovs', hash='5g9CmyIyoXquM5TCKik6Kw', space_id=1, created_by_id=1, run_id=3, created_at=<django.db.models.expressions.DatabaseDefault object at 0x3695ef290>)